In [13]:
%pip install ipywidgets -q
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.metrics.pairwise import cosine_similarity
import ipywidgets as widgets
from IPython.display import display, clear_output


Note: you may need to restart the kernel to use updated packages.


In [14]:
df = pd.read_csv('Dataset .csv')
df.info(), df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9551 entries, 0 to 9550
Data columns (total 21 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Restaurant ID         9551 non-null   int64  
 1   Restaurant Name       9551 non-null   object 
 2   Country Code          9551 non-null   int64  
 3   City                  9551 non-null   object 
 4   Address               9551 non-null   object 
 5   Locality              9551 non-null   object 
 6   Locality Verbose      9551 non-null   object 
 7   Longitude             9551 non-null   float64
 8   Latitude              9551 non-null   float64
 9   Cuisines              9542 non-null   object 
 10  Average Cost for two  9551 non-null   int64  
 11  Currency              9551 non-null   object 
 12  Has Table booking     9551 non-null   object 
 13  Has Online delivery   9551 non-null   object 
 14  Is delivering now     9551 non-null   object 
 15  Switch to order menu 

(None,
    Restaurant ID         Restaurant Name  Country Code              City  \
 0        6317637        Le Petit Souffle           162       Makati City   
 1        6304287        Izakaya Kikufuji           162       Makati City   
 2        6300002  Heat - Edsa Shangri-La           162  Mandaluyong City   
 3        6318506                    Ooma           162  Mandaluyong City   
 4        6314302             Sambo Kojin           162  Mandaluyong City   
 
                                              Address  \
 0  Third Floor, Century City Mall, Kalayaan Avenu...   
 1  Little Tokyo, 2277 Chino Roces Avenue, Legaspi...   
 2  Edsa Shangri-La, 1 Garden Way, Ortigas, Mandal...   
 3  Third Floor, Mega Fashion Hall, SM Megamall, O...   
 4  Third Floor, Mega Atrium, SM Megamall, Ortigas...   
 
                                      Locality  \
 0   Century City Mall, Poblacion, Makati City   
 1  Little Tokyo, Legaspi Village, Makati City   
 2  Edsa Shangri-La, Ortigas, Manda

In [15]:
df['Cuisines'] = df['Cuisines'].fillna('Other')

In [16]:
df['Cuisines'] = df['Cuisines'].apply(lambda x: x.split(',')[0].strip())

features = df[['Cuisines', 'Price range', 'Aggregate rating', 'Votes', 'Average Cost for two', 'City']].copy()

encoder = OneHotEncoder(sparse_output=False)
encoded_features = encoder.fit_transform(features[['Cuisines', 'Price range', 'City']])
encoded_features = pd.DataFrame(encoded_features, columns=encoder.get_feature_names_out(['Cuisines', 'Price range', 'City']))

In [17]:
scaler = MinMaxScaler()
rating_scaled = scaler.fit_transform(features[['Aggregate rating']])

In [18]:
combined_features = np.hstack([encoded_features, rating_scaled])

restaurant_names = df['Restaurant Name'].reset_index(drop=True)

combined_features.shape, restaurant_names.head()

((9551, 266),
 0          Le Petit Souffle
 1          Izakaya Kikufuji
 2    Heat - Edsa Shangri-La
 3                      Ooma
 4               Sambo Kojin
 Name: Restaurant Name, dtype: object)

In [ ]:
# Add this mapping
price_bins = {
    1: (0, 200),
    2: (200, 500),
    3: (500, 1000),
    4: (1000, float('inf'))
}
def recommend_restaurants(cuisine_preference, budget_min, budget_max, city_preference, top_n=5):
    # Filter based on price range
    filtered_df = df[
        (df['Average Cost for two'] >= budget_min) & 
        (df['Average Cost for two'] <= budget_max) &
        (df['City'] == city_preference)
    ]

    if filtered_df.empty:
        return pd.DataFrame({'Message': ['No restaurants match your preferences.']})

    filtered_encoded = encoder.transform(filtered_df[['Cuisines', 'Price range', 'City']])
    filtered_ratings = scaler.transform(filtered_df[['Aggregate rating']])
    filtered_features = np.hstack([filtered_encoded, filtered_ratings])

    user_encoded = encoder.transform([[cuisine_preference, 2, city_preference]]) 
    avg_rating = np.array([[0.5]])
    user_profile = np.hstack([user_encoded, avg_rating])

    similarity_scores = cosine_similarity(user_profile, filtered_features)[0]
    top_indices = similarity_scores.argsort()[-top_n:][::-1]
    
    # Create a copy of the filtered dataframe with selected indices
    result_df = filtered_df.iloc[top_indices].copy()
    
    # Add similarity scores to the result dataframe
    result_df['Similarity Score'] = similarity_scores[top_indices]
    # Format similarity scores as percentage with 2 decimal points
    result_df['Similarity Score'] = result_df['Similarity Score'].apply(lambda x: f"{x:.2%}")
    
    return result_df[['Restaurant Name', 'Cuisines', 'Average Cost for two', 'Aggregate rating', 'Similarity Score']]



In [20]:
cuisine_widget = widgets.Dropdown(
    options=sorted(df['Cuisines'].dropna().unique()),
    description='Cuisine:',
    style={'description_width': 'initial'}
)

city_widget = widgets.Dropdown(
    options=sorted(df['City'].dropna().unique()),
    description='City:',
    style={'description_width': 'initial'}
)

price_slider = widgets.IntRangeSlider(
    value=[100, 500],
    min=0,
    max=2000,
    step=50,
    description='Price Range (₹):',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='70%')
)

submit_button = widgets.Button(
    description='Get Recommendations',
    button_style='success',
    tooltip='Click to get recommendations'
)

output = widgets.Output()


In [21]:
def on_submit(b):
    with output:
        clear_output()
        cuisine = cuisine_widget.value
        city = city_widget.value
        min_price, max_price = price_slider.value

        print(f"Showing recommendations for {cuisine} cuisine in {city} (₹{min_price}–₹{max_price})")
        display(recommend_restaurants(cuisine, min_price, max_price, city))
submit_button.on_click(on_submit)

In [22]:
display(widgets.VBox([
    cuisine_widget,
    city_widget,
    price_slider,
    submit_button,
    output
]))


In [ ]:
# Fix the function call with correct parameters
recommend_restaurants('Chinese', 0, 1000, 'Bangalore')


c:\Users\bossg\miniconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Restaurant Name,Cuisines,Average Cost for two,Aggregate rating,Similarity Score
0,Eat Street,North Indian,400,4.3,0.411032
